In [ ]:
!nvidia-smi

Sun Mar 30 16:03:13 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.86.15              Driver Version: 570.86.15      CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   66C    P0             22W /   72W |       1MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!rm -r "nllb-200-distilled-1.3B-en-hi"
!rm -r "nllb-200-distilled-600M-en-hi"
!rm -r "nllb-IWSLT2025-en-hi"

rm: cannot remove 'nllb-200-distilled-1.3B-en-hi': No such file or directory
rm: cannot remove 'nllb-200-distilled-600M-en-hi': No such file or directory
rm: cannot remove 'nllb-IWSLT2025-en-hi': No such file or directory


In [ ]:
%%capture
!pip install transformers -q
!pip install datasets -q
!pip install evaluate -q
!pip install jiwer -q
!pip install sacrebleu -q
!pip install peft -q

In [ ]:
from huggingface_hub import notebook_login, whoami

In [ ]:
notebook_login()

In [ ]:
print(whoami())

{'type': 'user', 'id': '61728a033edf4cc38a81237a', 'name': 'joanitolopo', 'fullname': 'Joanito Agili Lopo', 'email': 'amalopo99@gmail.com', 'emailVerified': True, 'canPay': False, 'periodEnd': 1743465599, 'isPro': False, 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/1652231681579-61728a033edf4cc38a81237a.jpeg', 'orgs': [{'type': 'org', 'id': '643e51141d0e956d94baaade', 'name': 'kreasof-ai', 'fullname': 'Kreasof AI', 'email': None, 'canPay': False, 'periodEnd': None, 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/63b6f2e752c02ae8acbaa4d8/r_B-tRS854Qo5EOURHxL4.png', 'roleInOrg': 'write', 'isEnterprise': False}, {'type': 'org', 'id': '65129257d4556f9e6504afed', 'name': 'SEACrowd', 'fullname': 'SEACrowd', 'email': 'seacrowd.research@gmail.com', 'canPay': False, 'periodEnd': None, 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/616cd2e040e2f69baa1c7af2/n6b0McWLIRdksum9KE90i.png', 'roleInOrg': 'contributor', 'isEnterpris

In [ ]:
import os
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from datasets import load_dataset, Dataset, Audio, DatasetDict
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
import evaluate
from datasets import config
import torch
from datasets import IterableDatasetDict
from peft import (
    TaskType,
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_kbit_training,
)
import transformers
import numpy as np

In [ ]:
# Set the whole variables use
# ======== Model Argument ============
model_name_or_path = "facebook/nllb-200-distilled-600M"
load_in_8bit = False
use_lora = False
lora_r = 32
lora_alpha = 64
lora_dropout = 0.05
lora_target_modules= ["q_proj", "v_proj"]
bias = "none"

# ======== Data Argument ============
data_path = "kreasof-ai/en-hi-IWSLT2025"
source_language = "eng_Latn"
target_language = "hin_Deva"

# ======== Training Argument ============
push_to_hub = True
hf_repo = "kreasof-ai"
output_dir = f"{model_name_or_path.split('/')[1]}-en2hi"
do_normalize_eval = True
do_lower_case = False
do_remove_punctuation = False
hf_repo = "kreasof-ai"

cache_dir = "/content/huggingface_cache"
os.makedirs(cache_dir, exist_ok=True)

# Set ALL Hugging Face related cache directories
os.environ["TRANSFORMERS_CACHE"] = os.path.join(cache_dir, "transformers")
os.environ["HF_DATASETS_CACHE"] = os.path.join(cache_dir, "datasets")
os.environ["HF_HOME"] = os.path.join(cache_dir, "hf_home")
os.environ["HF_ASSETS_CACHE"] = os.path.join(cache_dir, "assets")
os.environ["HUGGINGFACE_HUB_CACHE"] = os.path.join(cache_dir, "hub")
os.environ["HF_MODULES_CACHE"] = os.path.join(cache_dir, "modules")

# Create all directories
for dir_path in [os.environ["TRANSFORMERS_CACHE"],
                os.environ["HF_DATASETS_CACHE"],
                os.environ["HF_HOME"],
                os.environ["HF_ASSETS_CACHE"],
                os.environ["HUGGINGFACE_HUB_CACHE"],
                os.environ["HF_MODULES_CACHE"]]:
    os.makedirs(dir_path, exist_ok=True)

# Force datasets to use the new cache
config.HF_DATASETS_CACHE = os.environ["HF_DATASETS_CACHE"]

metric_bleu = evaluate.load("sacrebleu")
metric_chrf = evaluate.load("chrf")
metric_wer = evaluate.load("wer")

torch.set_float32_matmul_precision("high")
device = "cuda" if torch.cuda.is_available() else "cpu"

def load_model_and_tokenizer():
  tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
  model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path).to(device)

  tokenizer.src_lang = source_language
  tokenizer.tgt_lang = target_language

  return model, tokenizer

def prepare_dataset(tokenizer):
  raw_datasets = IterableDatasetDict()
  raw_datasets["train"] = load_dataset(data_path, split="train", streaming=True)
  raw_datasets["test"] = load_dataset(data_path, split="dev", streaming=True)

  raw_datasets["train"] = raw_datasets["train"].remove_columns(["audio"])
  raw_datasets["test"] = raw_datasets["test"].remove_columns(["audio"])

  raw_datasets["train"] = raw_datasets["train"].filter(lambda x: x["text_en"] is not None and x["text_hi"] is not None)
  raw_datasets["test"] = raw_datasets["test"].filter(lambda x: x["text_en"] is not None and x["text_hi"] is not None)

  def tokenize_function(batch):
    input = batch["text_en"]
    target = batch["text_hi"]

    # encode target text to label ids
    model_inputs = tokenizer(batch["text_en"],
                                max_length=128,
                                text_target=batch["text_hi"],
                                truncation=True,
                                padding="max_length")
    return model_inputs

  dataset = raw_datasets.map(tokenize_function, remove_columns=list(next(iter(raw_datasets.values())).features)).with_format("torch")
  return dataset["train"], dataset["test"]

def setup_model_for_training(model):
  if load_in_8bit:
    model = prepare_model_for_kbit_training(model)
  else:
    model.enable_input_require_grads()

  # if use_accelerator:
  #   model = accelerator.prepare(model)

  if use_lora:
    lora_config = LoraConfig(
        r=lora_r,
        lora_alpha=lora_alpha,
        target_modules=lora_target_modules,
        lora_dropout=lora_dropout,
        bias=bias,
    )
    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()

  return model

def train():
  model, tokenizer = load_model_and_tokenizer()
  model = setup_model_for_training(model)
  train_dataset, test_dataset = prepare_dataset(tokenizer)
  data_collator = DataCollatorForSeq2Seq(tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True)

  def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    if isinstance(pred_ids, tuple):
      pred_ids = pred_ids[0]

    # Replace -100s used for padding as we can't decode them
    pred_ids = np.where(pred_ids != -100, pred_ids, tokenizer.pad_token_id)
    label_ids = np.where(label_ids != -100, label_ids, tokenizer.pad_token_id)

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric_wer.compute(predictions=pred_str, references=label_str)
    bleu = metric_bleu.compute(predictions=pred_str, references=label_str)
    bleu = round(bleu["score"], 2)
    chrf = metric_chrf.compute(predictions=pred_str, references=label_str)
    chrf = round(chrf["score"], 2)

    return {"bleu": bleu, "chrf": chrf, "wer": wer}

  training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=5000,
    gradient_checkpointing=True,
    bf16=True,
    eval_strategy="steps",
    per_device_eval_batch_size=4,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="chrf",
    greater_is_better=True,
    push_to_hub=True,
    hub_model_id=f"{hf_repo}/{output_dir}",
    num_train_epochs=3)

  trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer)

  if use_lora:
    old_state_dict = model.state_dict
    model.state_dict = (
        lambda self, *_, **__: get_peft_model_state_dict(
            self, old_state_dict()
        )
    ).__get__(model, type(model))

  transformers.logging.set_verbosity_info()
  trainer.train()
  trainer.push_to_hub()

train()

Resolving data files:   0%|          | 0/122 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/122 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/122 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/122 [00:00<?, ?it/s]

<ipython-input-7-8437d5977a9c>:166: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
***** Running training *****
  Num examples = 20,000
  Num Epochs = 9,223,372,036,854,775,807
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 5,000
  Number of trainable parameters = 615,073,792
/usr/local/lib/python3.10/dist-packages/transformers/data/data_collator.py:657: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
`use_cache=True` is incompatible with gradient checkpointing. Set

Step,Training Loss,Validation Loss,Bleu,Chrf,Wer
1000,2.543700,2.390966,30.900000,53.240000,78.966063
2000,0.180600,0.490757,29.370000,51.740000,79.665820


Streaming output truncated to the last 5000 lines.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Tra

Buffered data was truncated after reaching the output size limit.